[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Deyht/ML_OSAE_M2/blob/main/codes/CNN/ASIRRA_CIANNA.ipynb)

---


**Link to the CIANNA github repository**
https://github.com/Deyht/CIANNA

### **CIANNA installation**

#### Query GPU allocation and properties

If nvidia-smi fail, it might indicate that you launched the colab session whithout GPU reservation.  
To change the type of reservation go to "Runtime"->"Change runtime type" and select "GPU" as your hardware accelerator.

In [ ]:
%%shell

nvidia-smi

cd /content/

git clone https://github.com/NVIDIA/cuda-samples/

cd /content/cuda-samples/Samples/1_Utilities/deviceQuery/

make SMS="50 60 70 80"

./deviceQuery | grep Capability | cut -c50- > ~/cuda_infos.txt
./deviceQuery | grep "CUDA Driver Version / Runtime Version" | cut -c57- >> ~/cuda_infos.txt

cd ~/

If you are granted a GPU that supports high FP16 compute scaling (e.g the Tesla T4), it is advised to change the mixed_precision parameter in the last cell to "FP16C_FP32A".  
See the detail description on mixed precision support with CIANNA on the [Systeme Requirements](https://github.com/Deyht/CIANNA/wiki/1\)-System-Requirements) wiki page.

#### Clone CIANNA git repository

In [ ]:
%%shell

cd /content/

git clone https://github.com/Deyht/CIANNA

cd CIANNA

#### Compiling CIANNA for the allocated GPU generation

There is no guaranteed forward or backward compatibility between Nvidia GPU generation, and some capabilities are generation specific. For these reasons, CIANNA must be provided the platform GPU generation at compile time.
The following cell will automatically update all the necessary files based on the detected GPU, and compile CIANNA.

In [ ]:
%%shell

cd /content/CIANNA

mult="10"
cat ~/cuda_infos.txt
comp_cap="$(sed '1!d' ~/cuda_infos.txt)"
cuda_vers="$(sed '2!d' ~/cuda_infos.txt)"

lim="11.1"
old_arg=$(awk '{if ($1 < $2) print "-D CUDA_OLD";}' <<<"${cuda_vers} ${lim}")

sm_val=$(awk '{print $1*$2}' <<<"${mult} ${comp_cap}")

gen_val=$(awk '{if ($1 >= 80) print "-D GEN_AMPERE"; else if($1 >= 70) print "-D GEN_VOLTA";}' <<<"${sm_val}")

sed -i "s/.*arch=sm.*/\\t\tcuda_arg=\"\$cuda_arg -D CUDA -D comp_CUDA -lcublas -lcudart -arch=sm_$sm_val $old_arg $gen_val\"/g" compile.cp
sed -i "s/\/cuda-[0-9][0-9].[0-9]/\/cuda-$cuda_vers/g" compile.cp
sed -i "s/\/cuda-[0-9][0-9].[0-9]/\/cuda-$cuda_vers/g" src/python_module_setup.py

./compile.cp CUDA PY_INTERF

mv src/build/lib.linux-x86_64-* src/build/lib.linux-x86_64

#### Testing CIANNA installation

**IMPORTANT NOTE**   
CIANNA is mainly used in a script fashion and was not designed to run in notebooks. Every cell code that directly invokes CIANNA functions must be run as a script to avoid possible errors.  
To do so, the cell must have the following structure.

```
%%shell

cd /content/CIANNA

python3 - <<EOF

[... your python code ...]

EOF
```

This syntax allows one to easily edit python code in the notebook while running the cell as a script. Note that all the notebook variables can not be accessed by the cell in this context.


In [ ]:
%%shell

cd /content/

wget https://share.obspm.fr/s/jqGotokdYDXTbDS/download/data_asirra.tar.gz

tar -xzf data_asirra.tar.gz

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from PIL import Image
from PIL import ImageOps
from tqdm import tqdm

def make_square(im, min_size=128, fill_color=(0, 0, 0, 0)):
    x, y = im.size
    size = max(min_size, x, y)
    new_im = Image.new('RGB', (size, size), fill_color)
    new_im.paste(im, (int((size - x) / 2), int((size - y) / 2)))
    return new_im

def zoom_at(img, x, y, zoom):
    w, h = img.size
    zoom2 = zoom*2
    img = img.crop((x - w / zoom2, y - h / zoom2,
                    x + w / zoom2, y + h / zoom2))
    return img.resize((w, h), Image.LANCZOS)

def roll_zeropad(a, shift, axis=None):
    a = np.asanyarray(a)
    if shift == 0: return a
    if axis is None:
        n = a.size
        reshape = True
    else:
        n = a.shape[axis]
        reshape = False
    if np.abs(shift) > n:
        res = np.zeros_like(a)
    elif shift < 0:
        shift += n
        zeros = np.zeros_like(a.take(np.arange(n-shift), axis))
        res = np.concatenate((a.take(np.arange(n-shift,n), axis), zeros), axis)
    else:
        zeros = np.zeros_like(a.take(np.arange(n-shift,n), axis))
        res = np.concatenate((zeros, a.take(np.arange(n-shift), axis)), axis)
    if reshape:
        return res.reshape(a.shape)
    else:
        return res

def im_transform(im):
	im2 = im.copy()
	if(np.random.randint(0,2)):
		im2 = ImageOps.mirror(im2)
	im2 = im2.transform(im2.size, Image.AFFINE, (1.0-0.2*np.random.rand(),0.0, (np.random.rand()*2.0-1.0)*(im2.size[0]*0.1),
		0.0,1.0-0.2*np.random.rand(), (np.random.rand()*2.0-1.0)*(im2.size[1]*0.1)))
	#im2 = im2.rotate(np.random.rand()*40.0-20.0)
	im2 = im2.resize((128,128))

	return im2


orig_nb_images = 11500
test_size = 1000
image_size = 128
augm_fact = 1

train_im = np.zeros((orig_nb_images*augm_fact*2, 3, image_size, image_size), dtype="uint8")

for i in tqdm(range(0, orig_nb_images)):
	im = Image.open("/content/data/PetImages/Cat/"+str(i)+".jpg")
	width, height = im.size

	im = make_square(im)
	width2, height2 = im.size

	x_offset = int((width2 - width)*0.5)
	y_offset = int((height2 - height)*0.5)

	im = im.resize((image_size,image_size))

	for k in range(0,augm_fact):
		im_array = np.asarray(im_transform(im))
		for depth in range(0,3):
			train_im[i*augm_fact+k,depth,:,:] = im_array[:,:,depth]

for i in tqdm(range(0,orig_nb_images)):
	im = Image.open("/content/data/PetImages/Dog/"+str(i)+".jpg")
	width, height = im.size

	im = make_square(im)
	width2, height2 = im.size

	x_offset = int((width2 - width)*0.5)
	y_offset = int((height2 - height)*0.5)

	im = im.resize((image_size,image_size))

	for k in range(0,augm_fact):
		im_array = np.asarray(im_transform(im))
		for depth in range(0,3):
			train_im[orig_nb_images*augm_fact+i*augm_fact+k,depth,:,:] = im_array[:,:,depth]

train_im.tofile("train_im.dat")

del (train_im)


test_im = np.zeros((test_size*2, 3, image_size, image_size), dtype="uint8")

for i in tqdm(range(0, test_size)):
	im = Image.open("/content/data/PetImages/Cat/"+str(orig_nb_images+i)+".jpg")
	width, height = im.size

	im = make_square(im)
	width2, height2 = im.size

	x_offset = int((width2 - width)*0.5)
	y_offset = int((height2 - height)*0.5)

	im = im.resize((image_size,image_size))

	im_array = np.asarray((im))
	for depth in range(0,3):
		test_im[i,depth,:,:] = im_array[:,:,depth]

for i in tqdm(range(0,test_size)):
	im = Image.open("/content/data/PetImages/Dog/"+str(orig_nb_images+i)+".jpg")
	width, height = im.size

	im = make_square(im)
	width2, height2 = im.size

	x_offset = int((width2 - width)*0.5)
	y_offset = int((height2 - height)*0.5)

	im = im.resize((image_size,image_size))

	im_array = np.asarray((im))
	for depth in range(0,3):
		test_im[test_size+i,depth, :,:] = im_array[:,:,depth]

test_im.tofile("test_im.dat")

del (test_im)

In [ ]:
%%shell

python3 - << EOF

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageOps

import sys
sys.path.insert(0,'/content/CIANNA/src/build/lib.linux-x86_64')
import CIANNA as cnn

def i_ar(int_list):
	return np.array(int_list, dtype="int")

def f_ar(float_list):
	return np.array(float_list, dtype="float32")

cnn.init(in_dim=i_ar([128,128]), in_nb_ch=3, out_dim=2, bias=0.1, b_size=16,
	comp_meth="C_CUDA", dynamic_load=1, mixed_precision="FP32C_FP32A")

image_size = 128
nb_per_class = 11500
nb_test = 1000
augm_fact = 1
train_data = np.fromfile("train_im.dat", dtype="uint8")
train_data = np.reshape(train_data, ((nb_per_class*2*augm_fact, image_size*image_size*3))).astype("float32")

train_targets = np.zeros((nb_per_class*2*augm_fact,2), dtype="float32")

train_data[:,:] /= 255.0

train_targets[0:nb_per_class*augm_fact,0] = 1.0
train_targets[nb_per_class*augm_fact:,1]  = 1.0

cnn.create_dataset("TRAIN", nb_per_class*augm_fact*2, train_data[:,:], train_targets[:,:])
del (train_data, train_targets)

test_data = np.fromfile("test_im.dat", dtype="uint8")
test_data = np.reshape(test_data, ((nb_test*2, image_size*image_size*3))).astype("float32")

test_targets = np.zeros((nb_test*2,2), dtype="float32")

test_data[:,:] /= 255.0

test_targets[0:nb_test,0] = 1.0
test_targets[nb_test:,1]  = 1.0

cnn.create_dataset("VALID", nb_test*2, test_data[:,:], test_targets[:,:])
cnn.create_dataset("TEST",  nb_test*2, test_data[:,:], test_targets[:,:])

del (test_data, test_targets)



cnn.conv(f_size=i_ar([3,3]), nb_filters=16, padding=i_ar([1,1]), activation="RELU")
cnn.pool(p_size=i_ar([2,2]), p_type="MAX")
cnn.conv(f_size=i_ar([3,3]), nb_filters=32, padding=i_ar([1,1]), activation="RELU")
cnn.pool(p_size=i_ar([2,2]), p_type="MAX")
cnn.conv(f_size=i_ar([3,3]), nb_filters=64, padding=i_ar([1,1]), activation="RELU")
cnn.pool(p_size=i_ar([2,2]), p_type="MAX")
cnn.conv(f_size=i_ar([3,3]), nb_filters=128, padding=i_ar([1,1]), activation="RELU")
cnn.pool(p_size=i_ar([2,2]), p_type="MAX")
cnn.dense(nb_neurons=512, activation="RELU", drop_rate=0.5)
cnn.dense(nb_neurons=2, activation="SMAX")

for t in range(0,1):
	cnn.train(nb_iter=40, learning_rate=0.002, end_learning_rate=0.00004, control_interv=1,
		momentum=0.9, lr_decay=0.06, confmat=1, shuffle_gpu=0, save_every=20, shuffle_every=1)

	if(t == 0):
		cnn.perf_eval

EOF